In [1]:
import pickle

from mmcv.runner import get_dist_info, init_dist, load_checkpoint
from pyskl.models import build_model
from pyskl.datasets import build_dataloader, build_dataset

import mmcv
import os
import os.path as osp
import time
import torch
import torch.distributed as dist
from mmcv import Config, load
from mmcv.cnn import fuse_conv_bn
from mmcv.engine import multi_gpu_test
from mmcv.fileio.io import file_handlers
from mmcv.parallel import MMDistributedDataParallel
from mmcv.runner import get_dist_info, init_dist, load_checkpoint

from pyskl.datasets import build_dataloader, build_dataset
from pyskl.models import build_model
from pyskl.utils import cache_checkpoint, mc_off, mc_on, test_port

In [2]:
config_file = "D:\\FYP\\HAR-ZSL-XAI\\pyskl\\configs\\posec3d\\slowonly_r50_346_k400\\limb.py"
cfg = Config.fromfile(config_file)

if not hasattr(cfg, 'dist_params'):
    cfg.dist_params = dict(backend='nccl')

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load model

In [3]:
checkpoint_path = "../model_saves/Pose3D/kinetic_400_346_limb.pth"

In [4]:
arguments = {
    "average_clips":None,
    "checkpoint":checkpoint_path,
    "fuse_conv_bn":True,
    "tmpdir":"temp_dir",
    "launcher":"pytorch",
    "local_rank":0
}

In [5]:
if 'LOCAL_RANK' not in os.environ:
    os.environ['LOCAL_RANK'] = str(arguments["local_rank"])

if "RANK" not in os.environ:
    os.environ['RANK'] = str(0)
    os.environ['WORLD_SIZE'] = str(1)

In [6]:
model = build_model(cfg.model)
load_checkpoint(model, checkpoint_path, map_location='cpu')

load checkpoint from local path: ../model_saves/Pose3D/kinetic_400_346_limb.pth


OrderedDict([('backbone.conv1.conv.weight',
              tensor([[[[[-1.2584e-02, -8.3466e-04, -6.1006e-03,  ..., -8.4638e-03,
                          -2.8386e-02, -1.5192e-02],
                         [ 3.3752e-02,  1.5777e-02,  3.9949e-02,  ..., -1.9431e-02,
                           1.1950e-03,  2.3671e-02],
                         [-2.6003e-02, -1.8944e-03,  6.0830e-02,  ..., -3.8321e-02,
                          -1.7503e-02, -1.4342e-02],
                         ...,
                         [-2.6298e-02, -3.7876e-02,  8.3003e-03,  ..., -4.1446e-03,
                          -3.4105e-02, -8.6852e-04],
                         [-1.4345e-02, -3.2849e-02, -6.6792e-02,  ..., -4.1586e-02,
                           1.2901e-02, -2.5720e-03],
                         [ 1.9790e-04, -6.3391e-02, -1.0212e-01,  ...,  4.8647e-02,
                           6.3595e-02,  1.0857e-02]]],
              
              
                       [[[-1.5862e-02, -2.1877e-02, -2.6346e-02,  ...,  

In [7]:
model

Recognizer3D(
  (backbone): ResNet3dSlowOnly(
    (conv1): ConvModule(
      (conv): Conv3d(17, 32, kernel_size=(1, 7, 7), stride=(1, 1, 1), padding=(0, 3, 3), bias=False)
      (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck3d(
        (conv1): ConvModule(
          (conv): Conv3d(32, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
          (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=T

In [8]:
model.backbone.layer3

Sequential(
  (0): Bottleneck3d(
    (conv1): ConvModule(
      (conv): Conv3d(256, 128, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
      (bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (conv2): ConvModule(
      (conv): Conv3d(128, 128, kernel_size=(1, 3, 3), stride=(2, 2, 2), padding=(0, 1, 1), bias=False)
      (bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (conv3): ConvModule(
      (conv): Conv3d(128, 512, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn): BatchNorm3d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (downsample): ConvModule(
      (conv): Conv3d(256, 512, kernel_size=(1, 1, 1), stride=(2, 2, 2), bias=False)
      (bn): BatchNorm3d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): ReLU(i

In [9]:
torch.rand([1, 20, 17, 48, 64, 64]).shape

torch.Size([1, 20, 17, 48, 64, 64])

In [13]:
model = model.to(device)

In [14]:
model(torch.rand([1, 20, 17, 48, 64, 64]).to(device),return_loss=False)

OutOfMemoryError: CUDA out of memory. Tried to allocate 480.00 MiB (GPU 0; 6.00 GiB total capacity; 4.25 GiB already allocated; 0 bytes free; 4.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

init_dist(arguments["launcher"], **cfg.dist_params)
rank, world_size = get_dist_info()
cfg.gpu_ids = range(world_size)

## Pretrained model inference

In [8]:
out = "Pose3DResults"

eval_cfg = cfg.get('evaluation', {})
keys = ['interval', 'tmpdir', 'start', 'save_best', 'rule', 'by_epoch', 'broadcast_bn_buffers']
for key in keys:
    eval_cfg.pop(key, None)

eval_cfg['metrics'] = ['top_k_accuracy', 'mean_class_accuracy']
os.makedirs(out,exist_ok=True)

# set cudnn benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
cfg.data.test.test_mode = True

if not hasattr(cfg, 'dist_params'):
    cfg.dist_params = dict(backend='nccl')

default_mc_cfg = ('localhost', 22077)
memcached = cfg.get('memcached', False)

def inference_recognizer(__model, data_point, outputs=None, as_tensor=True, **kwargs):
    """Inference a video with the recognizer.

    Args:
        __model (nn.Module): The loaded recognizer.
        data_point (str | dict | ndarray): The video file path / url or the
            rawframes directory path / results dictionary (the input of
            pipeline) / a 4D array T x H x W x 3 (The input video).
        outputs (list(str) | tuple(str) | str | None) : Names of layers whose
            outputs need to be returned, default: None.
        as_tensor (bool): Same as that in ``OutputHook``. Default: True.

    Returns:
        dict[tuple(str, float)]: Top-5 recognition result dict.
        dict[torch.tensor | np.ndarray]:
            Output feature maps from layers specified in `outputs`.
            :param outputs:
            :param data_point:
            :param __model:
    """

    # forward the model
    # with OutputHook(model, outputs=outputs, as_tensor=as_tensor) as h:
    # model.cls_head.register_forward_hook(get_activation('dropout'))
    # with torch.no_grad():
    #     result = model(return_loss=False, **data)
    with torch.no_grad():
        my_output = None

        def my_hook(module_, input_, output_):
            nonlocal my_output
            my_output = output_

        #a_hook = model.backbone.layer3.register_forward_hook(my_hook)
        a_hook = __model.cls_head.fc_cls.register_forward_hook(my_hook)
        __model(return_loss=False, **data)
        a_hook.remove()
        # return my_output
    #     returned_features = h.layer_outputs if outputs else None

    # num_classes = scores.shape[-1]
    # score_tuples = tuple(zip(range(num_classes), scores))
    # score_sorted = sorted(score_tuples, key=itemgetter(1), reverse=True)

    # top5_label = score_sorted[:5]
    # if outputs:
    #     return top5_label, returned_features
    return my_output.squeeze()

In [29]:
def inference_pytorch(args, __model, __cfg, d):
    """Get predictions by pytorch models."""
    if args["average_clips"] is not None:
        # You can set average_clips during testing, it will override the
        # original setting
        if __cfg.model.get('test_cfg') is None and __cfg.get('test_cfg') is None:
            __cfg.model.setdefault('test_cfg',
                                 dict(average_clips=args["average_clips"]))
        else:
            if __cfg.model.get('test_cfg') is not None:
                __cfg.model.test_cfg.average_clips = args["average_clips"]
            else:
                __cfg.test_cfg.average_clips = args["average_clips"]

    # build the model and load checkpoint
    __model = build_model(__cfg.model)

    if args["checkpoint"] is None:
        work_dir = __cfg.work_dir
        args["checkpoint"] = osp.join(work_dir, 'latest.pth')
        assert osp.exists(args["checkpoint"])

    args["checkpoint"] = cache_checkpoint(args["checkpoint"])
    load_checkpoint(__model, args["checkpoint"], map_location='cpu')

    activation = {}
    def get_activation(name):
        def hook(__model, __input, output):
            activation[name] = output.detach()
        return hook

    if args["fuse_conv_bn"]:
        __model = fuse_conv_bn(__model)

    for batch_no, data in enumerate(data_loader):
        print(data["imgs"].shape,data["label"])
        __model.backbone.layer3.register_forward_hook(get_activation('layer3'))
        output = __model(data)
        print(output,activation)
    return outputs

In [30]:
from torch.utils.data import DataLoader

# build the dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = DataLoader(dataset)

2023-02-27 13:42:08,676 - pyskl - INFO - 173 videos remain after valid thresholding


In [31]:
dataset

In [32]:
#for i,d in enumerate(data_loader):
#    print((d["imgs"]>0).sum(),type(d["label"]),i)

In [33]:
outputs = inference_pytorch(arguments,model, cfg, data_loader)

load checkpoint from local path: ../model_saves/Pose3D/kinetic_400_346_limb.pth
torch.Size([1, 20, 17, 48, 64, 64]) ['ascending stairs']


ValueError: Label should not be None.